In [ ]:
# ============================================================
# 🚀 Realtime Voice Activity Detection (VAD) and Speaker Diarization
# Author: Mr.Jack (https://github.com/Mr-Jack-Tung)
# Date: 2025-11-02
# Description: Hướng dẫn xây dựng hệ thống VAD (Voice Activity Detection) và nhận diện người nói (Speaker Diarization) thời gian thực sử dụng Python và các thư viện mã nguồn mở ("pyannote/speaker-diarization").
# ============================================================

## Installation

In [1]:
# Cài đặt ffmpeg trên MacOS bằng homebrew
!brew install -q ffmpeg

In [2]:
!pip install -q numpy sounddevice webrtcvad soundfile

In [3]:
!pip install -q --upgrade pip

In [4]:
!pip install -q setuptools

In [5]:
!pip install -q openai-whisper

In [6]:
# Cài đặt thêm torch để hỗ trợ pyannote.audio
!pip install -q torch torchaudio

In [7]:
# Install pyannote.audio for speaker diarization
!pip install -q pyannote.audio

In [8]:
!pip install -q resemblyzer scikit-learn

## Imports & Implementation

In [ ]:
# Import required libraries
import queue
import threading
import numpy as np
import sounddevice as sd
from dataclasses import dataclass
from typing import List, Optional, Dict
from collections import deque
import webrtcvad
import time
import whisper
import soundfile as sf
import os

# pyannote.audio may be optional; import safely
try:
    from pyannote.audio import Pipeline
    print("✓ pyannote.audio imported successfully")
except Exception as e:
    Pipeline = None
    print(f"! pyannote.audio not available: {e}")

# Optional local diarization libraries (resemblyzer + sklearn)
try:
    from resemblyzer import VoiceEncoder
    from sklearn.metrics.pairwise import cosine_similarity
    print("✓ resemblyzer and sklearn imported successfully")
except Exception as e:
    VoiceEncoder = None
    cosine_similarity = None
    print(f"! local diarization not available: {e} - run: pip install resemblyzer scikit-learn")

@dataclass
class SpeechSegment:
    samples: np.ndarray
    start_time: float
    end_time: float
    confidence: float
    transcript: str = ""
    speaker_id: str = ""  # Added speaker identification

In [11]:
class RealtimeVAD:
    def __init__(self, 
                 sample_rate: int = 16000,
                 chunk_duration_ms: int = 30,
                 padding_duration_ms: int = 300,
                 silence_duration_ms: int = 500,
                 vad_sensitivity: int = 3):
        """
        Args:
            sample_rate: Phải là 16000 cho WebRTC VAD
            chunk_duration_ms: Độ dài mỗi chunk (30ms là tối ưu cho WebRTC VAD)
            padding_duration_ms: Padding trước và sau speech
            silence_duration_ms: Thời gian silence để kết thúc segment
            vad_sensitivity: Độ nhạy của VAD (1-3, 3 là nhạy nhất)
        """
        self.sample_rate = sample_rate
        self.chunk_duration_ms = chunk_duration_ms
        self.chunk_size = int(sample_rate * chunk_duration_ms / 1000)
        self.padding_duration_ms = padding_duration_ms
        self.silence_duration_ms = silence_duration_ms
        
        # Initialize WebRTC VAD
        self.vad = webrtcvad.Vad()
        self.vad.set_mode(vad_sensitivity)

        # Buffers
        self.audio_buffer = deque(maxlen=32000)  # 2 seconds buffer
        self.current_speech = []
        self.speech_segments = queue.Queue()
        
        # State tracking
        self.in_speech = False
        self.silence_start = None
        self.speech_start = None
        self.processed_samples = 0

        # Recording storage (full session) - will be written to WAV at the end (fallback)
        self.recorded_samples = []  # store int16 samples

        # Optional writer queue (started by higher-level class if streaming to disk)
        self.audio_write_queue = None

    #2. Audio Input Handler:
    def _audio_callback(self, indata, frames, time, status):
        """Callback for sounddevice's InputStream"""
        if status:
            print(f"Status: {status}")
            
        # Convert to mono and correct format (int16)
        audio_chunk = (indata[:, 0] * 32767).astype(np.int16)
        
        # Add to processing queue for VAD
        self.audio_buffer.extend(audio_chunk)

        # If an audio writer queue exists, push the chunk for streaming write
        if getattr(self, 'audio_write_queue', None) is not None:
            try:
                # put a copy so further mutations don't affect queued data
                self.audio_write_queue.put(audio_chunk.copy(), block=False)
            except Exception:
                # queue full or other issue - drop silently to avoid blocking audio thread
                pass
        
        # Process complete chunks
        while len(self.audio_buffer) >= self.chunk_size:
            chunk = np.array([self.audio_buffer.popleft() 
                            for _ in range(self.chunk_size)])
            self._process_chunk(chunk)

    #3. VAD Processing Logic:
    def _process_chunk(self, chunk: np.ndarray):
        """Process một chunk audio với VAD"""
        # Append to full recording buffer (keep as Python ints to avoid large numpy memory until write)
        try:
            self.recorded_samples.extend(chunk.tolist())
        except Exception:
            # Fallback: if extend fails for memory reasons, silently continue (still do VAD)
            pass

        is_speech = self.vad.is_speech(chunk.tobytes(), self.sample_rate)
        current_time = self.processed_samples / self.sample_rate

        if is_speech and not self.in_speech:
            # Speech bắt đầu
            self.in_speech = True
            self.speech_start = max(0, current_time - self.padding_duration_ms/1000)
            self.silence_start = None
            
            # Add padding từ buffer trước đó
            padding_samples = int(self.padding_duration_ms * self.sample_rate / 1000)
            if self.audio_buffer:
                padding = list(self.audio_buffer)[-padding_samples:]
                self.current_speech.extend(padding)
            
        elif not is_speech and self.in_speech:
            # Potential end of speech
            if self.silence_start is None:
                self.silence_start = current_time
                
            # Check if silence đủ dài
            if (current_time - self.silence_start) > self.silence_duration_ms/1000:
                self._finalize_speech_segment()
                
        # Collect samples nếu đang trong speech segment
        if self.in_speech:
            self.current_speech.extend(chunk)
            
        self.processed_samples += len(chunk)

    def _finalize_speech_segment(self):
        """Kết thúc speech segment hiện tại"""
        if not self.current_speech:
            return
            
        # Add post-speech padding
        padding_samples = int(self.padding_duration_ms * self.sample_rate / 1000)
        if self.audio_buffer:
            padding = list(self.audio_buffer)[:padding_samples]
            self.current_speech.extend(padding)
        
        # Create speech segment
        segment = SpeechSegment(
            samples=np.array(self.current_speech),
            start_time=self.speech_start,
            end_time=self.processed_samples / self.sample_rate,
            confidence=0.9
        )
        
        # Add to output queue
        self.speech_segments.put(segment)
        
        # Reset state
        self.current_speech = []
        self.in_speech = False
        self.speech_start = None
        self.silence_start = None

    #4. Main Recording Loop:
    def start(self, duration: Optional[float] = None):
        """Bắt đầu recording và VAD processing"""
        try:
            with sd.InputStream(channels=1,
                            samplerate=self.sample_rate,
                            dtype=np.float32,
                            callback=self._audio_callback,
                            blocksize=self.chunk_size):
                
                print(f"Started VAD recording (Press Ctrl+C to stop)")
                
                start_time = time.time()
                while True:
                    # Check duration
                    if duration and (time.time() - start_time) > duration:
                        break
                        
                    # Process available speech segments
                    try:
                        segment = self.speech_segments.get_nowait()
                        self._process_speech_segment(segment)
                    except queue.Empty:
                        time.sleep(0.1)
                        continue
                        
        except KeyboardInterrupt:
            print("\nRecording stopped.")
            
    def _process_speech_segment(self, segment: SpeechSegment):
        """Process detected speech segment - Override trong subclass"""
        duration = segment.end_time - segment.start_time
        print(f"Speech detected: {duration:.2f}s ({segment.start_time:.2f}s - {segment.end_time:.2f}s)")
        # Có thể thêm xử lý khác ở đây (e.g., transcription)


In [12]:
class TranscriptionWorker:
    def __init__(self, model_name: str = "base"):
        self.model = whisper.load_model(model_name)
        self.queue = queue.Queue()
        self.results = queue.Queue()
        self.running = True
        self.thread = threading.Thread(target=self._process_loop)
        self.thread.daemon = True
        self.thread.start()
    
    def _process_loop(self):
        while self.running:
            try:
                segment = self.queue.get(timeout=1)
                
                # Save temporary WAV file for Whisper
                temp_file = f"temp_{time.time()}.wav"
                sf.write(temp_file, segment.samples, 16000)
                
                try:
                    # Transcribe with Whisper using FP32
                    result = self.model.transcribe(temp_file, fp16=False)
                    segment.transcript = result["text"].strip()
                    self.results.put(segment)
                finally:
                    # Cleanup temp file
                    if os.path.exists(temp_file):
                        os.remove(temp_file)
                        
            except queue.Empty:
                continue
            except Exception as e:
                print(f"Transcription error: {e}")
    
    def add_segment(self, segment: SpeechSegment):
        self.queue.put(segment)
    
    def get_transcribed_segment(self, timeout: float = 0.1) -> Optional[SpeechSegment]:
        try:
            return self.results.get(timeout=timeout)
        except queue.Empty:
            return None
    
    def stop(self):
        self.running = False
        if self.thread.is_alive():
            self.thread.join()

In [13]:
# Example với realtime transcription và speaker diarization
class TranscribingVAD(RealtimeVAD):
    def __init__(self, *args, transcript_file="transcript.txt", audio_file="recording.wav", 
                 max_meeting_duration: Optional[float] = None, 
                 use_diarization: bool = True,
                 auth_token: Optional[str] = None,  # Add auth token parameter
                 **kwargs):
        """Transcribing VAD with speaker diarization.

        Args:
            transcript_file: path to write transcripts
            audio_file: path to save full recording
            max_meeting_duration: optional maximum meeting duration in seconds
            use_diarization: whether to enable speaker diarization
            auth_token: Hugging Face auth token for pyannote.audio
        """
        super().__init__(*args, **kwargs)
        self.transcriber = TranscriptionWorker(model_name="base")
        self.transcript_file = transcript_file
        self.audio_file = audio_file
        self.pending_transcripts = 0  # Đếm số transcript đang chờ xử lý
        self.max_meeting_duration = max_meeting_duration
        # We'll attempt pyannote if requested and token provided; otherwise may fallback to local diarizer
        self.use_diarization = use_diarization and auth_token is not None
        self.auth_token = auth_token
        
        # Diarization state
        self.pipeline = None
        self.local_diarizer = None
        self.diarization_cache = {}  # Cache diarization results by segment
        self.min_segment_duration = 2.0  # Minimum segment duration for reliable diarization
        self.long_segments = []  # Store longer segments for batch processing
        
        # Initialize speaker diarization if enabled
        if self.use_diarization:
            try:
                print("Initializing speaker diarization pipeline (pyannote)...")
                # Prefer huggingface_hub.login when available
                try:
                    from huggingface_hub import login
                except Exception:
                    login = None

                if self.auth_token:
                    if login is not None:
                        print("Logging into Hugging Face hub for diarization...")
                        login(self.auth_token)
                    else:
                        # Fallback: set env var for the process
                        os.environ["HUGGINGFACE_HUB_TOKEN"] = self.auth_token

                # Attempt to load pyannote pipeline, retry with revision='main' if necessary
                from pyannote.audio import Pipeline as _Pipeline
                try:
                    self.pipeline = _Pipeline.from_pretrained("pyannote/speaker-diarization")
                except Exception as e:
                    msg = str(e)
                    if "Revisions must be passed" in msg or "revision" in msg:
                        print("Caught revision-related error; retrying with revision='main'...")
                        try:
                            self.pipeline = _Pipeline.from_pretrained("pyannote/speaker-diarization", revision="main")
                        except Exception as e2:
                            raise RuntimeError(f"Failed to load pyannote pipeline with revision fallback: {e2}")
                    else:
                        raise

                print("Speaker diarization pipeline initialized successfully")
            except Exception as e:
                print(f"Warning: Speaker diarization initialization failed: {e}")
                print("Falling back to local diarizer if available (resemblyzer)...")
                self.pipeline = None
                # Try to initialize local diarizer if possible
                try:
                    if VoiceEncoder is not None:
                        self.local_diarizer = LocalDiarizer(sample_rate=self.sample_rate, similarity_threshold=0.70)
                        print("Local diarizer (resemblyzer) initialized successfully")
                        # set flag to indicate diarization is enabled via local method
                        self.use_diarization = True
                    else:
                        print("Local diarizer not available (resemblyzer missing), disabling diarization")
                        self.use_diarization = False
                except Exception as e2:
                    print(f"Failed to initialize local diarizer: {e2}")
                    self.use_diarization = False
                    self.local_diarizer = None
        else:
            # If diarization not requested or no auth token, try local diarizer if installed
            if VoiceEncoder is not None:
                try:
                    self.local_diarizer = LocalDiarizer(sample_rate=self.sample_rate, similarity_threshold=0.70)
                    print("Local diarizer (resemblyzer) initialized for offline diarization")
                    self.use_diarization = True
                except Exception as e:
                    print(f"Local diarizer init failed: {e}")
                    self.local_diarizer = None
            else:
                self.local_diarizer = None

        # Create transcript file with header
        with open(self.transcript_file, 'w', encoding='utf-8') as f:
            f.write("=== Transcript File ===\n")
            f.write(f"Created: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            if self.max_meeting_duration:
                f.write(f"Max meeting duration (s): {self.max_meeting_duration}\n")
            f.write(f"Speaker diarization: {'Enabled' if self.use_diarization else 'Disabled'}\n")
            if not self.use_diarization and use_diarization:
                f.write("Note: Diarization requested but disabled due to missing auth token or failure to initialize\n")
            f.write("====================\n\n")

        # writer thread state
        self.audio_write_queue = None
        self._audio_writer_thread = None
        self._audio_writer_running = False
        
        # Cached speaker segments for diarization
        self._speaker_segments = {}  # timestamp -> speaker_id
        self._last_diarization_time = 0
        self._diarization_interval = 30  # Process every 30 seconds
    
    def _identify_speaker(self, audio_segment: np.ndarray, start_time: float, end_time: float) -> str:
        """Identify speaker for an audio segment.
        If pyannote pipeline is available, use it; otherwise use the local diarizer.
        """
        if not self.use_diarization:
            return "UNKNOWN"

        # Prefer pyannote pipeline if present
        if self.pipeline is not None:
            try:
                temp_file = f"temp_diarize_{time.time()}.wav"
                sf.write(temp_file, audio_segment, self.sample_rate)
                diarization = self.pipeline(temp_file)
                speaker_id = "UNKNOWN"
                max_overlap = 0
                for turn, _, speaker in diarization.itertracks(yield_label=True):
                    overlap = min(turn.end, end_time) - max(turn.start, start_time)
                    if overlap > max_overlap:
                        speaker_id = speaker
                        max_overlap = overlap
                try:
                    os.remove(temp_file)
                except Exception:
                    pass
                return speaker_id or "UNKNOWN"
            except Exception as e:
                print(f"pyannote identification failed for segment: {e}")
                # fall through to local diarizer if available

        if self.local_diarizer is not None:
            try:
                return self.local_diarizer.identify(audio_segment)
            except Exception as e:
                print(f"Local diarizer failed: {e}")
                return "UNKNOWN"

        return "UNKNOWN"

    def _process_speech_segment(self, segment: SpeechSegment):
        """Process a speech segment: identify speaker, transcribe, and save results"""
        try:
            # Get speaker ID
            speaker_id = self._identify_speaker(segment.samples, segment.start_time, segment.end_time)
            segment.speaker_id = speaker_id
            
            # Add to transcription queue
            self.transcriber.add_segment(segment)
            self.pending_transcripts += 1
            
            # Check for completed transcripts
            while True:
                result = self.transcriber.get_transcribed_segment(timeout=0.1)
                if result is None:
                    break
                    
                self.pending_transcripts -= 1
                
                # Write to transcript file
                duration = result.end_time - result.start_time
                with open(self.transcript_file, 'a', encoding='utf-8') as f:
                    f.write(f"\n[{result.start_time:.1f}s -> {result.end_time:.1f}s] {result.speaker_id}: {result.transcript}")
                
                # Print to console
                print(f"\n[{duration:.1f}s] {result.speaker_id}: {result.transcript}")
                
        except Exception as e:
            print(f"Error processing segment: {e}")

In [14]:
class LocalDiarizer:
    def __init__(self, sample_rate: int = 16000, similarity_threshold: float = 0.75):
        """Initialize a local diarizer using resemblyzer voice embeddings.
        
        Args:
            sample_rate: Audio sample rate (must match input)
            similarity_threshold: Cosine similarity threshold for speaker matching (0.0-1.0)
                                Lower values make it easier to detect different speakers
        """
        self.sample_rate = sample_rate
        self.similarity_threshold = similarity_threshold
        self.speaker_embeddings = {}  # speaker_id -> embedding
        self.next_speaker_id = 1
        
        # Load resemblyzer encoder
        try:
            from resemblyzer import VoiceEncoder
            self.encoder = VoiceEncoder()
            print("✓ Local diarizer: loaded resemblyzer encoder")
        except Exception as e:
            raise RuntimeError(f"Failed to initialize resemblyzer encoder: {e}")
    
    def embed(self, audio: np.ndarray) -> np.ndarray:
        """Generate embedding for an audio segment."""
        # resemblyzer expects float32 in [-1, 1]
        if audio.dtype == np.int16:
            audio = audio.astype(np.float32) / 32767.0
        return self.encoder.embed_utterance(audio)
    
    def identify(self, audio: np.ndarray) -> str:
        """Identify the speaker in an audio segment.
        Returns speaker ID (e.g., "SPEAKER_1") or creates a new one if no match.
        """
        # Get embedding for this segment
        try:
            embedding = self.embed(audio)
        except Exception as e:
            print(f"Failed to generate embedding: {e}")
            return "UNKNOWN"
            
        # If no speakers yet, create first one
        if not self.speaker_embeddings:
            speaker_id = f"SPEAKER_{self.next_speaker_id}"
            self.speaker_embeddings[speaker_id] = embedding
            self.next_speaker_id += 1
            return speaker_id
            
        # Compare with existing speakers
        max_similarity = -1
        best_speaker = None
        
        for speaker_id, spk_embedding in self.speaker_embeddings.items():
            similarity = cosine_similarity(embedding.reshape(1, -1), spk_embedding.reshape(1, -1))[0, 0]
            
            if similarity > max_similarity:
                max_similarity = similarity
                best_speaker = speaker_id
                
        # If similarity is high enough, use existing speaker
        if max_similarity >= self.similarity_threshold:
            # Update speaker embedding with running average
            old_emb = self.speaker_embeddings[best_speaker]
            self.speaker_embeddings[best_speaker] = 0.9 * old_emb + 0.1 * embedding
            return best_speaker
            
        # Otherwise create new speaker
        new_speaker = f"SPEAKER_{self.next_speaker_id}"
        self.speaker_embeddings[new_speaker] = embedding
        self.next_speaker_id += 1
        return new_speaker
    
    def reset(self):
        """Clear speaker history/embeddings."""
        self.speaker_embeddings.clear()
        self.next_speaker_id = 1

In [ ]:
!export HUGGINGFACE_HUB_TOKEN=""

## Test

In [16]:
# Test thử pyannote.audio pipeline (with revision fallback and local-diarizer fallback)
def test_diarization():
    try:
        print("Testing diarization pipeline (pyannote preferred, local fallback)...")

        # Ensure VoiceEncoder is available in this scope (in case the Imports cell wasn't executed)
        try:
            VoiceEncoder  # reference to see if exists
        except NameError:
            try:
                from resemblyzer import VoiceEncoder as _VoiceEncoder
                VoiceEncoder = _VoiceEncoder
            except Exception:
                VoiceEncoder = None

        # Helper to attempt Pipeline.from_pretrained with a revision fallback
        def load_pyannote_pipeline():
            try:
                from pyannote.audio import Pipeline as _Pipeline
            except Exception as e:
                raise RuntimeError(f"pyannote.audio not available: {e}")

            # Ensure auth is set
            try:
                from huggingface_hub import login as _login
            except Exception:
                _login = None

            auth_token = os.environ.get("HUGGINGFACE_HUB_TOKEN") or ""
            if not auth_token:
                raise ValueError("Auth token not set. Set HUGGINGFACE_HUB_TOKEN environment variable or populate auth_token in the notebook.")

            if _login is not None:
                print("Logging into Hugging Face hub...")
                _login(auth_token)
            else:
                os.environ["HUGGINGFACE_HUB_TOKEN"] = auth_token

            # Try loading pipeline; on specific revision error retry with revision='main'
            try:
                return _Pipeline.from_pretrained("pyannote/speaker-diarization")
            except Exception as e:
                msg = str(e)
                if "Revisions must be passed" in msg or "revision" in msg:
                    print("Caught revision-related error from from_pretrained; retrying with revision='main'...")
                    try:
                        return _Pipeline.from_pretrained("pyannote/speaker-diarization", revision="main")
                    except Exception as e2:
                        raise RuntimeError(f"Failed to load pyannote pipeline with revision='main': {e2}")
                else:
                    raise

        pipeline = None
        try:
            pipeline = load_pyannote_pipeline()
            print("✓ pyannote pipeline initialized successfully")
            use_pyannote = True
        except Exception as e:
            print(f"pyannote pipeline initialization failed: {e}")
            use_pyannote = False

        # Create a test audio file (two distinct tones -> two speakers)
        import numpy as np
        import soundfile as sf
        sample_rate = 16000
        duration = 5
        t = np.linspace(0, duration, sample_rate * duration)
        speaker1 = np.sin(2 * np.pi * 440 * t[:len(t)//2]) * 0.5
        speaker2 = np.sin(2 * np.pi * 880 * t[len(t)//2:]) * 0.5
        audio = np.concatenate([speaker1, np.zeros(int(0.1 * sample_rate)), speaker2])
        noise = np.random.normal(0, 0.01, len(audio))
        audio = audio + noise
        test_file = "diarization_test.wav"
        sf.write(test_file, audio, sample_rate)

        if use_pyannote:
            print("Running pyannote diarization on test file...")
            diarization = pipeline(test_file)
            print("Diarization results (pyannote):")
            count = 0
            total = 0.0
            for turn, _, speaker in diarization.itertracks(yield_label=True):
                print(f"[{turn.start:.1f}s -> {turn.end:.1f}s] {speaker}")
                count += 1
                total += turn.end - turn.start
            if count == 0:
                raise RuntimeError("pyannote produced no speaker turns")
            print(f"✓ pyannote diarization ok: {count} turns, {total:.1f}s total")
            return True
        else:
            # Fallback to local diarizer
            if VoiceEncoder is None:
                raise RuntimeError("Neither pyannote available nor resemblyzer installed. Install resemblyzer for local diarization: pip install resemblyzer sklearn")
            print("Falling back to local resemblyzer-based diarizer for test...")
            ld = LocalDiarizer(sample_rate=sample_rate, similarity_threshold=0.65)
            # split test audio into two segments and identify
            half = len(audio)//2
            seg1 = (audio[:half]).astype(np.float32)
            seg2 = (audio[half:]).astype(np.float32)
            id1 = ld.identify((seg1 * 32767).astype(np.int16))
            id2 = ld.identify((seg2 * 32767).astype(np.int16))
            print(f"Local diarizer IDs: seg1={id1}, seg2={id2}")
            if id1 == id2:
                print("Warning: local diarizer assigned same speaker to both segments; consider lowering similarity_threshold")
            else:
                print("✓ Local diarizer produced different speaker IDs for the two test segments")
            return True

    except Exception as e:
        print(f"\n✗ Diarization test failed: {e}")
        print("Troubleshooting tips:")
        print(" - Ensure torch/torchaudio are installed and compatible")
        print(" - If using pyannote, ensure HUGGINGFACE_HUB_TOKEN is set and you've accepted the model license on huggingface.co")
        print(" - To use the local diarizer, install resemblyzer: pip install resemblyzer sklearn")
        return False

    finally:
        if os.path.exists(test_file):
            try:
                os.remove(test_file)
            except Exception:
                pass

# Run the test
print("Starting diarization system test...")
diarization_ok = test_diarization()

Starting diarization system test...
Testing diarization pipeline (pyannote preferred, local fallback)...
pyannote pipeline initialization failed: Auth token not set. Set HUGGINGFACE_HUB_TOKEN environment variable or populate auth_token in the notebook.
Falling back to local resemblyzer-based diarizer for test...
Loaded the voice encoder model on cpu in 0.01 seconds.
✓ Local diarizer: loaded resemblyzer encoder
Local diarizer IDs: seg1=SPEAKER_1, seg2=SPEAKER_2
✓ Local diarizer produced different speaker IDs for the two test segments
Local diarizer IDs: seg1=SPEAKER_1, seg2=SPEAKER_2
✓ Local diarizer produced different speaker IDs for the two test segments


## RUN

In [ ]:
# Create VAD instance with diarization if test passed
if diarization_ok:
    print("Creating TranscribingVAD instance...")
    vad = TranscribingVAD(
        transcript_file="meeting_transcript.txt",
        audio_file="meeting_audio.wav",
        max_meeting_duration=60,  # 60 second test
        use_diarization=True,
        auth_token=os.environ.get("HUGGINGFACE_HUB_TOKEN", "")
    )

Creating TranscribingVAD instance...
Initializing speaker diarization pipeline (pyannote)...
Logging into Hugging Face hub for diarization...
Initializing speaker diarization pipeline (pyannote)...
Logging into Hugging Face hub for diarization...
Caught revision-related error; retrying with revision='main'...
Caught revision-related error; retrying with revision='main'...
Falling back to local diarizer if available (resemblyzer)...
Loaded the voice encoder model on cpu in 0.01 seconds.
✓ Local diarizer: loaded resemblyzer encoder
Local diarizer (resemblyzer) initialized successfully
Falling back to local diarizer if available (resemblyzer)...
Loaded the voice encoder model on cpu in 0.01 seconds.
✓ Local diarizer: loaded resemblyzer encoder
Local diarizer (resemblyzer) initialized successfully


In [19]:
# Only proceed if diarization test passed
if diarization_ok:
    print("Starting VAD with diarization...")
    try:
        vad.start(duration=60)
    except KeyboardInterrupt:
        print("\nRecording stopped by user")
    except Exception as e:
        print(f"\nError during recording: {e}")
    finally:
        # Cleanup
        if hasattr(vad, 'transcriber'):
            vad.transcriber.stop()
else:
    print("Skipping run due to diarization test failure. Please fix the issues above first.")

Starting VAD with diarization...
Started VAD recording (Press Ctrl+C to stop)

[2.2s] SPEAKER_2: BOT HIGH BABOT

[2.2s] SPEAKER_2: BOT HIGH BABOT
